# Segmenting and Clustering Neighborhoods in Toronto

Install BeautifulSoup 4 

In [1]:
!pip install beautifulsoup4
!pip install geocoder

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

2. Use soup to get table with postcodes and also remove rows where Borough is not assigned *

In [3]:
soup = BeautifulSoup(req.content,'lxml')

table = soup.find('table', attrs={'class':'wikitable sortable'})

table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)


df = pd.DataFrame(res, columns=["Postcode", "Borough", "Neighbourhood"])
df = df.astype(str)

df.shape



(288, 3)

In [4]:
boroughs = df[df.Borough != 'Not assigned']
boroughs.shape

(211, 3)

* Replace all values where Neighbourhood == 'Not assigned' with Borough Name *

In [5]:
boroughs.Neighbourhood[boroughs.Neighbourhood == 'Not assigned'] = boroughs.Borough

boroughs.shape

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/jupyterlab/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


(211, 3)

* Merge Neighbourhood from same Borough*

In [6]:

finalBdf = boroughs.groupby(['Postcode','Borough']).agg(', '.join).reset_index()
finalBdf.shape

(103, 3)

*** As google geoCoder APIs are not reliable using csv file available. Please have the csv file 'Geospatial_Coordinates.csv' in same foler as jyp notebook ***

In [12]:

geoCoord=pd.read_csv('Geospatial_Coordinates.csv')

geoCoord.rename(index=str, columns={"Postal Code": "Postcode"}, inplace=True)


dfinal = boroughs.merge(geoCoord, on="Postcode", how = 'inner')

dfinal.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353
